In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from umap import UMAP
from sklearn.metrics import confusion_matrix, accuracy_score
import matplotlib.pyplot as plt

from scipy.stats import zscore
from scipy.io import loadmat, savemat
import mat73
import os
import time

# conda activate shinlab # installed umap here
# DON'T FORGET: to use synology, need to run the following line in terminal
# net use S: \\shinlab\ShinLab
print('DON''T FORGET: to use synology, need to run in terminal: \"net use S: \\shinlab\ShinLab\" ')


DONT FORGET: to use synology, need to run in terminal: "net use S: \shinlab\ShinLab" 


In [3]:
def semi_supervised_umap_knn(data, labels, probedata, n_neighbors=20, n_components=2, n_splits=10, opt_supervise_umap=True, random_state=42):
    """
    Perform semi-supervised UMAP with kNN decoding, incorporating negative sampling and advanced validation.

    Parameters:
        data (numpy.ndarray): High-dimensional input data. NtrialsXNneurons
        labels (numpy.ndarray): Labels for supervised UMAP training. NtrialsX1
        n_neighbors (int): Number of neighbors for UMAP and kNN.
        n_components (int): Number of UMAP dimensions.
        test_size (float): Fraction of data for testing.
        random_state (int): Random state for reproducibility.

    Returns:
        dict: Results including accuracy, confusion matrix, and embeddings.
    """

    opt_meancenter_embeddings = False    
    
    # todo: preprocess (mean-centering/z-scoring) based on train set
    X_probe = probedata
    
    # # Split data into training and test sets
    # X_train, X_test, y_train, y_test = train_test_split(
    #     data, labels, test_size=1.0/n_splits, random_state=random_state)

    # # Cross-validation setup
    # kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
    # Create StratifiedKFold object
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    train_index_list = []
    test_index_list = []
    y_test_list = []    
    accuracies = []
    confusion_matrices = []
    
    train_embeddings_list = []
    test_embeddings_list = []
    probe_embeddings_list = []
    
    y_train_pred_list = []
    y_test_pred_list = []
    y_probe_pred_list = []
    
    for train_index, test_index in skf.split(data, labels):
        X_train, X_test = data[train_index], data[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
            
        # Apply supervised UMAP with adjusted hyperparameters
        umap_model = UMAP(
            n_neighbors=n_neighbors,
            n_components=n_components,
            metric="cosine", # note, this makes mean-centering unnecessary
            output_metric="euclidean",
            learning_rate=1.0,
            init="spectral",
            min_dist=0.1,
            spread=1.0,
            repulsion_strength=1.0,
            negative_sample_rate=5,
            target_metric="categorical",
            dens_lambda=2.0,
            dens_frac=0.3,
            dens_var_shift=0.1,
            random_state=random_state
        )
        
        if opt_supervise_umap:
            # make umap supervised
            embeddings = umap_model.fit_transform(X_train, y=y_train)
        else:
            embeddings = umap_model.fit_transform(X_train)
    
        # to make umap unsupervised, use code below instead
        # embeddings = umap_model.fit_transform(X_train)
        
        if opt_meancenter_embeddings:
            train_embeddings = embeddings.copy()
            train_embeddings = train_embeddings - np.mean(embeddings, axis=0)
        else:
            train_embeddings = embeddings.copy()
            
        # Train kNN classifier on UMAP embeddings
        knn = KNeighborsClassifier(n_neighbors=n_neighbors)
        knn.fit(train_embeddings, y_train)
    
        # Transform test data using the same UMAP model
        test_embeddings = umap_model.transform(X_test)
    
        if opt_meancenter_embeddings:
            test_embeddings = test_embeddings - np.mean(embeddings, axis=0)
        
        # Predict train/test labels
        y_train_pred = knn.predict(train_embeddings)
        y_test_pred = knn.predict(test_embeddings)
    
            
        # probe data
        probe_embeddings = umap_model.transform(X_probe)
        if opt_meancenter_embeddings:
            probe_embeddings = probe_embeddings - np.mean(embeddings, axis=0)
        y_probe_pred = knn.predict(probe_embeddings)
        
        # Compute accuracy and confusion matrix
        accuracy = accuracy_score(y_test, y_test_pred)
        conf_matrix = confusion_matrix(y_test, y_test_pred)

        
        train_index_list.append(train_index)
        test_index_list.append(test_index)    
        y_test_list.append(y_test)    
        accuracies.append(accuracy)
        confusion_matrices.append(conf_matrix)
        
        train_embeddings_list.append(train_embeddings)
        test_embeddings_list.append(test_embeddings)
        probe_embeddings_list.append(probe_embeddings)
                
        y_train_pred_list.append(y_train_pred)
        y_test_pred_list.append(y_test_pred)
        y_probe_pred_list.append(y_probe_pred)
        
    return {
        # "umap_model": umap_model, # this raises error when trying to use savemat
        "train_index": train_index_list,
        "test_index": test_index_list,
        "test_truelabels": y_test_list,
        "accuracy": accuracies,
        "confusion_matrix": confusion_matrices,
        "train_embeddings": train_embeddings_list,
        "test_embeddings": test_embeddings_list,
        "probe_embeddings": probe_embeddings_list,
        "train_predlabels": y_train_pred_list,
        "test_predlabels": y_test_pred_list,
        "probe_predlabels": y_probe_pred_list,
    }


In [4]:
# inverted cross-validation (10-fold inverted cross validation is 10% training trials, 90% test trials)
def semi_supervised_umap_knn_invcrossval(data, labels, probedata, n_neighbors=20, n_components=2, n_splits=10, opt_supervise_umap=True, random_state=42):
    """
    Perform semi-supervised UMAP with kNN decoding, incorporating negative sampling and advanced validation.

    Parameters:
        data (numpy.ndarray): High-dimensional input data. NtrialsXNneurons
        labels (numpy.ndarray): Labels for supervised UMAP training. NtrialsX1
        n_neighbors (int): Number of neighbors for UMAP and kNN.
        n_components (int): Number of UMAP dimensions.
        test_size (float): Fraction of data for testing.
        random_state (int): Random state for reproducibility.

    Returns:
        dict: Results including accuracy, confusion matrix, and embeddings.
    """

    opt_meancenter_embeddings = False    
    
    # todo: preprocess (mean-centering/z-scoring) based on train set
    X_probe = probedata
    
    # # Split data into training and test sets
    # X_train, X_test, y_train, y_test = train_test_split(
    #     data, labels, test_size=1.0/n_splits, random_state=random_state)

    # # Cross-validation setup
    # kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
    # Create StratifiedKFold object
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    train_index_list = []
    test_index_list = []
    y_test_list = []    
    accuracies = []
    confusion_matrices = []
    
    train_embeddings_list = []
    test_embeddings_list = []
    probe_embeddings_list = []
    
    y_train_pred_list = []
    y_test_pred_list = []
    y_probe_pred_list = []
    
    for test_index, train_index in skf.split(data, labels): # NOTE this is the only line that changed from def semi_supervised_umap_knn
        X_train, X_test = data[train_index], data[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
            
        # Apply supervised UMAP with adjusted hyperparameters
        umap_model = UMAP(
            n_neighbors=n_neighbors,
            n_components=n_components,
            metric="cosine", # note, this makes mean-centering unnecessary
            output_metric="euclidean",
            learning_rate=1.0,
            init="spectral",
            min_dist=0.1,
            spread=1.0,
            repulsion_strength=1.0,
            negative_sample_rate=5,
            target_metric="categorical",
            dens_lambda=2.0,
            dens_frac=0.3,
            dens_var_shift=0.1,
            random_state=random_state
        )
        
        if opt_supervise_umap:
            # make umap supervised
            embeddings = umap_model.fit_transform(X_train, y=y_train)
        else:
            embeddings = umap_model.fit_transform(X_train)
    
        # to make umap unsupervised, use code below instead
        # embeddings = umap_model.fit_transform(X_train)
        
        if opt_meancenter_embeddings:
            train_embeddings = embeddings.copy()
            train_embeddings = train_embeddings - np.mean(embeddings, axis=0)
        else:
            train_embeddings = embeddings.copy()
            
        # Train kNN classifier on UMAP embeddings
        knn = KNeighborsClassifier(n_neighbors=n_neighbors)
        knn.fit(train_embeddings, y_train)
    
        # Transform test data using the same UMAP model
        test_embeddings = umap_model.transform(X_test)
    
        if opt_meancenter_embeddings:
            test_embeddings = test_embeddings - np.mean(embeddings, axis=0)
        
        # Predict train/test labels
        y_train_pred = knn.predict(train_embeddings)
        y_test_pred = knn.predict(test_embeddings)
    
            
        # probe data
        probe_embeddings = umap_model.transform(X_probe)
        if opt_meancenter_embeddings:
            probe_embeddings = probe_embeddings - np.mean(embeddings, axis=0)
        y_probe_pred = knn.predict(probe_embeddings)
        
        # Compute accuracy and confusion matrix
        accuracy = accuracy_score(y_test, y_test_pred)
        conf_matrix = confusion_matrix(y_test, y_test_pred)

        
        train_index_list.append(train_index)
        test_index_list.append(test_index)    
        y_test_list.append(y_test)    
        accuracies.append(accuracy)
        confusion_matrices.append(conf_matrix)
        
        train_embeddings_list.append(train_embeddings)
        test_embeddings_list.append(test_embeddings)
        probe_embeddings_list.append(probe_embeddings)
                
        y_train_pred_list.append(y_train_pred)
        y_test_pred_list.append(y_test_pred)
        y_probe_pred_list.append(y_probe_pred)
        
    return {
        # "umap_model": umap_model, # this raises error when trying to use savemat
        "train_index": train_index_list,
        "test_index": test_index_list,
        "test_truelabels": y_test_list,
        "accuracy": accuracies,
        "confusion_matrix": confusion_matrices,
        "train_embeddings": train_embeddings_list,
        "test_embeddings": test_embeddings_list,
        "probe_embeddings": probe_embeddings_list,
        "train_predlabels": y_train_pred_list,
        "test_predlabels": y_test_pred_list,
        "probe_predlabels": y_probe_pred_list,
    }


In [5]:
datadir = r'\\shinlab\ShinLab\OpenScopeData\00248_v240130\\'

# List all files and directories in the data directory
nwbdir = os.listdir(datadir)

nwbsessions = ['sub-619293', 'sub-619296', 'sub-620333', 'sub-620334', 
    'sub-625545', 'sub-625554', 'sub-625555', 'sub-630506', 
    'sub-631510', 'sub-631570', 'sub-633229', 'sub-637484'];

ises = 1 #'sub-619296'
pathpp = os.path.join(datadir, 'postprocessed', nwbsessions[ises], '')
print(ises, pathpp)

fn2load = pathpp + 'spkcnt_ICwcfg1_hireptt_V1RS_lmlv.mat'
try:
    spkcntlmlv = loadmat(fn2load)
except:
    spkcntlmlv = mat73.loadmat(fn2load)

sponfn2load = pathpp + 'psth_spontaneous_spikecount_V1RS.mat'
try:
    spkcntspon = loadmat(sponfn2load)
except:
    spkcntspon = mat73.loadmat(sponfn2load)


1 \\shinlab\ShinLab\OpenScopeData\00248_v240130\\postprocessed\sub-619296\


In [ ]:
# 2-dimensional embedding, 12 classes (hireptt): takes 10 min per session
Nsplits = 10
blocktrialorder = spkcntlmlv['trialorder'].flatten()
vislabel = blocktrialorder

Rblock = spkcntlmlv['spkcntorig']
Rspon = spkcntspon['psthsponspkcnt'].transpose()

tic = time.time()
UMAPKNN_semisup = semi_supervised_umap_knn(Rblock, vislabel, Rspon, n_components=2, n_splits=Nsplits, opt_supervise_umap=True)
UMAPKNN_unsup = semi_supervised_umap_knn(Rblock, vislabel, Rspon, n_components=2, n_splits=Nsplits, opt_supervise_umap=False)

UMAPKNN_semisup_invcv = semi_supervised_umap_knn_invcrossval(Rblock, vislabel, Rspon, n_components=2, n_splits=Nsplits, opt_supervise_umap=True)
UMAPKNN_unsup_invcv = semi_supervised_umap_knn_invcrossval(Rblock, vislabel, Rspon, n_components=2, n_splits=Nsplits, opt_supervise_umap=False)

toc = time.time() - tic
print('Elapsed time: ', toc)


UMAP_dict = {
    'UMAPKNN_semisup': UMAPKNN_semisup,
    'UMAPKNN_unsup': UMAPKNN_unsup,
    'UMAPKNN_semisup_invcv': UMAPKNN_semisup_invcv,
    'UMAPKNN_unsup_invcv': UMAPKNN_unsup_invcv,
}

fn2save = pathpp + 'UMAP_KNN_decoding_V1RS_spontaneous.mat'
savemat(fn2save, UMAP_dict)


In [6]:
# 2-dimensional embedding, 12 classes (hireptt): takes 10 min per session
Nsplits = 2
blocktrialorder = spkcntlmlv['trialorder'].flatten()
vislabel = blocktrialorder

Rblock = spkcntlmlv['spkcntorig']
Rspon = spkcntspon['psthsponspkcnt'].transpose()

tic = time.time()
UMAPKNN2_semisup = semi_supervised_umap_knn(Rblock, vislabel, Rspon, n_components=2, n_splits=Nsplits, opt_supervise_umap=True)
UMAPKNN2_unsup = semi_supervised_umap_knn(Rblock, vislabel, Rspon, n_components=2, n_splits=Nsplits, opt_supervise_umap=False)

toc = time.time() - tic
print('Elapsed time: ', toc)


UMAP_dict = {
    'UMAPKNN2_semisup': UMAPKNN2_semisup,
    'UMAPKNN2_unsup': UMAPKNN2_unsup,
}

fn2save = pathpp + 'UMAP_KNN_'+str(Nsplits)+'fold_decoding_V1RS_spontaneous.mat'
savemat(fn2save, UMAP_dict)


C:\ProgramData\anaconda3\envs\hs13\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\ProgramData\anaconda3\envs\hs13\lib\site-packages\umap\umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
C:\ProgramData\anaconda3\envs\hs13\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\ProgramData\anaconda3\envs\hs13\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\ProgramData\anaconda3\envs\hs13\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWar

Elapsed time:  147.0698697566986


TypeError: can only concatenate str (not "int") to str

In [ ]:
datadir = r'\\shinlab\ShinLab\OpenScopeData\00248_v240130\\'

# List all files and directories in the data directory
nwbdir = os.listdir(datadir)

nwbsessions = ['sub-619293', 'sub-619296', 'sub-620333', 'sub-620334', 
    'sub-625545', 'sub-625554', 'sub-625555', 'sub-630506', 
    'sub-631510', 'sub-631570', 'sub-633229', 'sub-637484'];

for ises in np.arange(len(nwbsessions)):
    pathpp = os.path.join(datadir, 'postprocessed', nwbsessions[ises], '')
    print(ises, pathpp)
    
    fn2load = pathpp + 'spkcnt_ICwcfg1_hireptt_V1RS_lmlv.mat'
    try:
        spkcntlmlv = loadmat(fn2load)
    except:
        spkcntlmlv = mat73.loadmat(fn2load)
    
    sponfn2load = pathpp + 'psth_spontaneous_spikecount_V1RS.mat'
    try:
        spkcntspon = loadmat(sponfn2load)
    except:
        spkcntspon = mat73.loadmat(sponfn2load)
    
    blocktrialorder = spkcntlmlv['trialorder'].flatten()
    vislabel = blocktrialorder
    
    Rblock = spkcntlmlv['spkcntorig']
    Rspon = spkcntspon['psthsponspkcnt'].transpose()
    
    Nsplits = 10
    UMAPdim_list = []
    for Ndim in range(1,11,1):
        tic = time.time()
        UMAPKNN_unsup = semi_supervised_umap_knn(Rblock, vislabel, Rspon, n_components=Ndim, n_splits=Nsplits, opt_supervise_umap=False)
        
        UMAPdim_list.append(UMAPKNN_unsup)
        toc = time.time() - tic
        print('Elapsed time: ', toc)
    
    
    UMAP_dict = {
        'embedding_dimensions': range(1,11,1),
        'UMAPKNN_dimensions': UMAPdim_list,
    }
    
    fn2save = pathpp + 'UMAP_KNN_decoding_V1RS_spontaneous_embeddims.mat'
    savemat(fn2save, UMAP_dict)

0 \\shinlab\ShinLab\OpenScopeData\00248_v240130\\postprocessed\sub-619293\


C:\ProgramData\anaconda3\envs\hs13\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\ProgramData\anaconda3\envs\hs13\lib\site-packages\umap\umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
C:\ProgramData\anaconda3\envs\hs13\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\ProgramData\anaconda3\envs\hs13\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\ProgramData\anaconda3\envs\hs13\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWar

Elapsed time:  147.64585399627686


C:\ProgramData\anaconda3\envs\hs13\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\ProgramData\anaconda3\envs\hs13\lib\site-packages\umap\umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
C:\ProgramData\anaconda3\envs\hs13\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\ProgramData\anaconda3\envs\hs13\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\ProgramData\anaconda3\envs\hs13\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWar

Elapsed time:  147.44637155532837


...

C:\ProgramData\anaconda3\envs\hs13\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\ProgramData\anaconda3\envs\hs13\lib\site-packages\umap\umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
C:\ProgramData\anaconda3\envs\hs13\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\ProgramData\anaconda3\envs\hs13\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\ProgramData\anaconda3\envs\hs13\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWar

Elapsed time:  177.4428415298462


C:\ProgramData\anaconda3\envs\hs13\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\ProgramData\anaconda3\envs\hs13\lib\site-packages\umap\umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
C:\ProgramData\anaconda3\envs\hs13\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\ProgramData\anaconda3\envs\hs13\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\ProgramData\anaconda3\envs\hs13\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWar

Elapsed time:  180.97404146194458


C:\ProgramData\anaconda3\envs\hs13\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\ProgramData\anaconda3\envs\hs13\lib\site-packages\umap\umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


In [ ]:
for Ndim in range(1,11,1):
    print(Ndim)